In [22]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
import time
from transformers import AutoModel
from transformers import AutoConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from calibrate_e_ml import calibrate_e_ml
from cg.node import construct_aggregation_graph

In [32]:
seq_len = 100
bs = 4
input_ids = torch.randint(1000, size=(bs, seq_len), dtype=torch.long, device='cpu')
model_name = "prajjwal1/bert-tiny"
cuda_exist = torch.cuda.is_available()
device = torch.device("cuda" if cuda_exist else "cpu")
information = calibrate_e_ml(model_name, bs, seq_len, device)

In [ ]:
input_ids.dtype

In [ ]:
information

In [25]:
level = information['linear'][0]
print(level)
fn = level['module']
fname=level['name']
fi_size, fi_dtype = level['inputs']
fo_size, fo_dtype = level['outputs']
fi = torch.rand(fi_size, dtype=fi_dtype)
fo = torch.rand(fo_size, dtype=fo_dtype)

fo_call = fn(fi)

{'name': 'encoder.layer.0.attention.self.query:Linear', 'module': Linear(in_features=128, out_features=128, bias=True), 'inputs': (torch.Size([4, 100, 128]), torch.float32), 'outputs': (torch.Size([4, 100, 128]), torch.float32), 'runtime': 0.012598949000000914}


In [26]:
f_start = time.perf_counter()
for _ in range(10000):
    fo_call = fn(fi)
f_end = time.perf_counter()
print(f_end-f_start)

4.047626644999809


In [27]:
trace = torch.jit.trace(fn, fi)
trace_graph = trace.inlined_graph
trace_graph

graph(%self : __torch__.torch.nn.modules.linear.___torch_mangle_1.Linear,
      %input : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu)):
  %2 : Tensor = prim::GetAttr[name="bias"](%self)
  %3 : Tensor = prim::GetAttr[name="weight"](%self)
  %4 : Float(128:1, 128:128, requires_grad=1, device=cpu) = aten::t(%3) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %output : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::matmul(%input, %4) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %6 : int = prim::Constant[value=1]() # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  %m_out : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::add_(%output, %2, %6) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  return (%m_out)

In [28]:
graph, op_data_types = construct_aggregation_graph(trace_graph, fname)

/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for prim::GetAttr!
  else:
/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for aten::t!
  else:
/Users/qqcao/work/DetecTor/cg/op_counter.py:127: UserWarning: no counter available for prim::Constant!
  else:


In [29]:
graph

Graph(name=encoder.layer.0.attention.self.query:Linear,
	inputs=[
		DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)
		DataNode(id=input, dtype=Float, shape=[4, 100, 128])],
	nodes=[
		OpNode(id=2, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)],
			outputs=[
				DataNode(id=2, dtype=None)],)
		OpNode(id=3, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_1.Linear)],
			outputs=[
				DataNode(id=3, dtype=None)],)
		OpNode(id=4, op=aten::t, 
			inputs=[
				DataNode(id=3, dtype=None)],
			outputs=[
				DataNode(id=4, dtype=Float, shape=[128, 128])],)
		OpNode(id=output, op=aten::matmul, flops=13107200, mem_bytes=671744, 
			inputs=[
				DataNode(id=input, dtype=Float, shape=[4, 100, 128])
				DataNode(id=4, dtype=Float, shape=[128, 128])],
			outputs=[
				DataNode(id=output, dtype=Float, shape=[4, 100, 128])],)
		OpNode(i

```
for each input size (bs, seq_len) pair:
    for each module:
        get jit trace, extract flops and mem_bytes,
        log (bs, seq_len, name, module level, flops, mem_bytes)
        run the module 100k times, log timestamp
        sleep 5s
        repeat 10 rounds
record energy with timestamp, log cpu_util, gpu_util, mem_util, gpu_util

extract data (bs, seq_len, name, module level, flops, mem_bytes, cpu_util, gpu_util, mem_util, gpu_util, energy)
calc energy variance, simple linear regression fitting
```   

In [30]:
from collections import defaultdict

def get_flops_mem_bytes(graph):
    flops = dict()
    mem_bytes = dict()

    for node in graph.nodes:
        flops[node.id] = node.flops
        mem_bytes[node.id] = node.mem_bytes
        if node.flops:
            print(node.op, node.flops, node.mem_bytes)
    return sum(flops.values()), sum(mem_bytes.values())

In [31]:
get_flops_mem_bytes(graph)

aten::matmul 13107200 671744
aten::add_ 51200 409600


(13158400, 1081344)

In [ ]:
from datetime import datetime
print(datetime.fromtimestamp(1605498402.136237))

In [118]:
import pandas
pandas.set_option('display.float_format', '{:.6f}'.format)

res= pandas.read_csv('data/mlexp/mlexp-linear-res2.csv')
energy= pandas.read_csv('data/mlexp/mlexp-linear-energy2.csv', error_bad_lines=False, usecols=[0, 2])

In [88]:
import json

model_name = 'bert-base-uncased'
level = 'linear'
runs = 10
bs = 16
seq_len = 384

prof_filename = f'{model_name}_{level}_r{runs}_b{bs}_i{seq_len}.json'

with open(f'data/mlexp/{prof_filename}') as f:
    prof_info = json.load(f)

In [89]:
prof_info[0]

{'name': 'encoder.layer.0.attention.self.query:Linear',
 'flops': 7252475904,
 'mem_bytes': 113246208,
 'start_1': 1610903590.6697361,
 'end_1': 1610903591.3599648,
 'start_2': 1610903592.3611808,
 'end_2': 1610903593.052057,
 'start_3': 1610903594.0532901,
 'end_3': 1610903594.7442207,
 'start_4': 1610903595.7448068,
 'end_4': 1610903596.435857,
 'start_5': 1610903597.4370837,
 'end_5': 1610903598.1282494,
 'start_6': 1610903599.1294367,
 'end_6': 1610903599.824273,
 'start_7': 1610903600.8254433,
 'end_7': 1610903601.5203035,
 'start_8': 1610903602.5215309,
 'end_8': 1610903603.216559,
 'start_9': 1610903604.2176173,
 'end_9': 1610903604.9122813,
 'start_10': 1610903605.913508,
 'end_10': 1610903606.6082392,
 'type': 'linear'}

In [119]:
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

energy=energy[energy['timestamp'].apply(lambda x: is_float(x))]
energy['value']=energy['value'].div(100)
energy['timestamp']=energy['timestamp'].astype(float)

In [129]:
res_np = res.to_numpy()
energy_np = energy.to_numpy()

In [131]:
res

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
0,1610903583.440226,2.600000,4.000000,2,0,0,15313,2,0,34326
1,1610903583.540479,1.800000,4.000000,2,0,0,15313,0,0,34343
2,1610903583.640747,0.100000,4.000000,2,0,0,15313,0,0,34491
3,1610903583.741015,0.100000,4.000000,2,0,0,15216,0,0,34587
4,1610903583.841277,0.100000,4.000000,2,0,0,15313,0,0,34491
...,...,...,...,...,...,...,...,...,...,...
1684,1610903752.278331,0.100000,4.000000,2,0,0,10150,0,0,6495
1685,1610903752.378598,0.100000,4.000000,2,0,0,10059,0,0,6400
1686,1610903752.478866,0.100000,4.000000,2,0,0,9861,0,0,6592
1687,1610903752.579133,0.100000,4.000000,2,0,0,10052,0,0,6397


In [132]:
res_t = res_np[:, 0]
energy_t = energy_np[:, 0]

In [133]:
energy_np

array([[1.61090358e+09, 1.12330000e+02],
       [1.61090358e+09,            nan],
       [1.61090358e+09, 1.00000000e-02],
       ...,
       [1.61090376e+09, 1.15410000e+02],
       [1.61090376e+09, 1.15140000e+02],
       [1.61090376e+09, 1.13470000e+02]])

In [134]:
import bisect

res_s = bisect.bisect_right(res_t, prof_info[0]['start_1'])
res_e = bisect.bisect_right(res_t, prof_info[0]['end_1'])

energy_s = bisect.bisect_right(energy_t, prof_info[0]['start_1'])
energy_e = bisect.bisect_right(energy_t, prof_info[0]['end_1'])

In [138]:
res[res_s:res_e]

,timestamp,cpu,mem,n_gpu,gpu,gpu_mem,gpu_power,gpu.1,gpu_mem.1,gpu_power.1
73,1610903590.760744,9.300000,9.000000,2,99,50,208903,0,0,34360
74,1610903590.861005,8.900000,9.000000,2,99,50,212446,0,0,34502
75,1610903590.961265,10.400000,9.000000,2,100,39,210800,0,0,34502
76,1610903591.061532,9.200000,9.000000,2,100,39,212381,0,0,34511
77,1610903591.161789,8.400000,9.000000,2,100,39,211546,0,0,34502
78,1610903591.262048,8.400000,9.000000,2,100,39,211469,0,0,34502


In [140]:
energy[energy_s:energy_e]['value'].sum()

1421.6399999999999

In [147]:
import numpy as np

for prof_item in prof_info:
    energy_runs = []
    for r in range(1, runs+1):
        energy_s = bisect.bisect_right(energy_t, prof_item[f'start_{r}'])
        energy_e = bisect.bisect_right(energy_t, prof_item[f'end_{r}'])
        energy_r = energy[energy_s:energy_e]['value'].sum()
        energy_runs.append(energy_r)
    std_percent = np.std(energy_runs) / np.mean(energy_runs)
    mean = np.mean(energy_runs)
    print(prof_item['type'], prof_item['name'], std_percent, mean)

linear encoder.layer.0.attention.self.query:Linear 0.07236582207171405 1242.308
linear encoder.layer.0.intermediate.dense:Linear 0.13399399857273184 8558.122
linear encoder.layer.0.output.dense:Linear 0.12190375616305438 7355.465000000001
linear pooler.dense:Linear 0.8961761566460925 93.628


In [180]:
import bisect
import json
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.float_format', '{:.6f}'.format)
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

model_name = 'bert-base-uncased'
level = 'level'

res= pd.read_csv(f'data/mlexp-linear-res-{model_name}.csv')
energy= pd.read_csv(f'data/mlexp-linear-energy-{model_name}.csv', error_bad_lines=False, usecols=[0, 2])
energy=energy[energy['value'].apply(lambda x: is_float(x))]
energy=energy[energy['timestamp'].apply(lambda x: is_float(x))]

energy['value']=energy['value'].astype(float).div(100)
energy['timestamp']=energy['timestamp'].astype(float)
    
res_np = res.to_numpy()
res_t = res_np[:, 0]
energy_np = energy.to_numpy()
energy_t = energy_np[:, 0]

# runs = 10
# bs = 16
# seq_len = 384


In [184]:
batch_sizes = []
seq_lens = []
stds = []
means = []
flops = []
names = []

for bs in list(range(2, 34, 2)) + [1]:
    for seq_len in range(16, 400, 16):
        prof_file = Path(f'data/mlexp/{model_name}_{level}_r{runs}_b{bs}_i{seq_len}.json')
        if not prof_file.exists():
            continue
        with open(prof_file) as f:
            prof_info = json.load(f)

        for prof_item in prof_info:
            energy_runs = []
            for r in range(1, runs+1):
                energy_s = bisect.bisect_right(energy_t, prof_item[f'start_{r}'])
                energy_e = bisect.bisect_right(energy_t, prof_item[f'end_{r}'])
                energy_r = energy[energy_s:energy_e]['value'].sum()
                energy_runs.append(energy_r)
            std_percent = np.std(energy_runs) / np.mean(energy_runs)*100
            mean = np.mean(energy_runs)
            batch_sizes.append(bs)
            seq_lens.append(seq_len)
            stds.append(std_percent)
            means.append(mean)
            flops.append(prof_item['flops']/1e6)
            names.append(prof_item['name'])

            print(f"b{bs},i{seq_len}:{prof_item['type']}, {std_percent:.1f}%, {mean/10:.2f} J, {prof_item['flops']/1e6:.2f} MFlops {prof_item['name']}")

b1,i288:linear, 5.4%, 157.86 J, 339.96 MFlops encoder.layer.0.attention.self.query:Linear
b1,i288:linear, 1.6%, 584.81 J, 1359.84 MFlops encoder.layer.0.intermediate.dense:Linear
b1,i288:linear, 1.5%, 568.11 J, 1359.18 MFlops encoder.layer.0.output.dense:Linear
b1,i288:linear, 14.0%, 55.62 J, 1.18 MFlops pooler.dense:Linear
b1,i304:linear, 7.0%, 154.86 J, 358.85 MFlops encoder.layer.0.attention.self.query:Linear
b1,i304:linear, 1.9%, 602.67 J, 1435.39 MFlops encoder.layer.0.intermediate.dense:Linear
b1,i304:linear, 1.9%, 565.54 J, 1434.69 MFlops encoder.layer.0.output.dense:Linear
b1,i304:linear, 11.7%, 53.54 J, 1.18 MFlops pooler.dense:Linear
b1,i320:linear, 4.5%, 162.12 J, 377.73 MFlops encoder.layer.0.attention.self.query:Linear
b1,i320:linear, 1.8%, 609.42 J, 1510.93 MFlops encoder.layer.0.intermediate.dense:Linear
b1,i320:linear, 2.0%, 576.70 J, 1510.20 MFlops encoder.layer.0.output.dense:Linear
b1,i320:linear, 8.2%, 56.25 J, 1.18 MFlops pooler.dense:Linear
b1,i336:linear, 4.2%, 1

In [185]:
info = pd.DataFrame(data={'b': batch_sizes, 'i': seq_lens, 'std': stds, 'mean': means, 'flops': flops, 'name': names})

In [186]:
pd.set_option('display.max_rows', 300)
info

,b,i,std,mean,flops,name
0,1,288,5.407070,1578.558000,339.959808,encoder.layer.0.attention.self.query:Linear
1,1,288,1.574682,5848.116000,1359.839232,encoder.layer.0.intermediate.dense:Linear
2,1,288,1.493060,5681.147000,1359.175680,encoder.layer.0.output.dense:Linear
3,1,288,13.993324,556.153000,1.179648,pooler.dense:Linear
4,1,304,7.024453,1548.598000,358.846464,encoder.layer.0.attention.self.query:Linear
5,1,304,1.941634,6026.749000,1435.385856,encoder.layer.0.intermediate.dense:Linear
6,1,304,1.905795,5655.402000,1434.685440,encoder.layer.0.output.dense:Linear
7,1,304,11.720155,535.446000,1.179648,pooler.dense:Linear
8,1,320,4.451367,1621.165000,377.733120,encoder.layer.0.attention.self.query:Linear
9,1,320,1.751715,6094.234000,1510.932480,encoder.layer.0.intermediate.dense:Linear


In [187]:
info.sort_values('std')

,b,i,std,mean,flops,name
21,1,368,1.107965,6368.135000,1737.572352,encoder.layer.0.intermediate.dense:Linear
18,1,352,1.444171,7095.814000,1661.214720,encoder.layer.0.output.dense:Linear
22,1,368,1.482529,7272.141000,1736.724480,encoder.layer.0.output.dense:Linear
2,1,288,1.493060,5681.147000,1359.175680,encoder.layer.0.output.dense:Linear
25,1,384,1.511401,6418.546000,1813.118976,encoder.layer.0.intermediate.dense:Linear
1,1,288,1.574682,5848.116000,1359.839232,encoder.layer.0.intermediate.dense:Linear
14,1,336,1.601675,7049.452000,1585.704960,encoder.layer.0.output.dense:Linear
13,1,336,1.687089,6182.481000,1586.479104,encoder.layer.0.intermediate.dense:Linear
9,1,320,1.751715,6094.234000,1510.932480,encoder.layer.0.intermediate.dense:Linear
6,1,304,1.905795,5655.402000,1434.685440,encoder.layer.0.output.dense:Linear
